In [ ]:
import sys
sys.path.append("/Users/anuram/Documents/hydrogenase-ftir/src")
sys.path.append("/Users/anuram/Documents/hydrogenase-ftir/notebooks/anu_winter_2024")
#sys.path.append("/Users/anuram/Documents/Documents/hydrogenase-ftir/src/hydrogenase_processing")

In [ ]:
%load_ext autoreload
%autoreload 2

#Setting Up and Importing the Necessary Packages/Libraries
##Package for reading in Bruker OPUS type files
from brukeropusreader import read_file
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
import pathlib

#Local Functions
from hydrogenase_processing.cut_range import cut_range_subtract_prospecpy_objects
from hydrogenase_processing.second_deriv import  second_deriv_prospecpy_objects
from hydrogenase_processing.baseline import baseline_correction_prospecpy_objects
from hydrogenase_processing.peak_fit import gaussian_fit_prospecpy_objects, lorentzian_fit_prospecpy_objects
from hydrogenase_processing.io import import_run_data
from hydrogenase_processing.interact import interact


# Importing spectra data and water vapor data

In [ ]:
#Paths to Local Data
path_to_water_vapor_data = pathlib.Path("../../data/opus_files/water_vapor")
path_to_output_plots_= pathlib.Path("../../data/output_plots/")
path_to_pD6_data = pathlib.Path("../../data/opus_files/pD6")

In [ ]:
pD6_raw_data = import_run_data(path_to_pD6_data, input_type="raw spectra", output_folder = '../../output_plots/')
wv_data = import_run_data(path_to_water_vapor_data)

In [ ]:
pD6_raw_data[0].output_folder

# Subtracting water vapor from raw spectra

In [ ]:
#Initializing dict of post water vapor subtraction spectra
cut_range_subtract_prospecpy_objects(pD6_raw_data, wv_data, range_start = 2150, range_end = 1850, showplots=False, verbose= True)
#pD6_raw_data[].cut_subtracted_data

## Get the second derivatives

In [ ]:
second_deriv_prospecpy_objects(pD6_raw_data, show_plots=False, verbose = True)

## Using a user interface please select the appropriate number of peaks using the threshold variable and number of anchor points using adjustment factor variable

In [ ]:
interact(pD6_raw_data,0.4,1.8)

# Baseline correction and cruve fit for an example spectra workflow:

## Fit baseline spline through these anchor points to be used for baseline correction of raw spectra

In [ ]:
baseline_correction_prospecpy_objects(pD6_raw_data, showplot = True, save = True, verbose = True)

# Fitting gaussian and lorentizian curves on the baseline corrected data

In [ ]:
gaussian_fit_prospecpy_objects(pD6_raw_data, show_plots = True, save = True, verbose = True)

In [ ]:
lorentzian_fit_prospecpy_objects(pD6_raw_data, show_plots = True, save = True, verbose = True)